In [ ]:
import sys
import os
import json
import tensorflow as tf

from ltv_utils import *
pd.set_option('display.float_format', '{:.4f}'.format)  # 保留10位小数，可调整
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


def parse_function(serialized_example):

    feature_description = {
        'deviceid': tf.io.FixedLenFeature([], tf.string),
        'install_date': tf.io.FixedLenFeature([], tf.string),
        'dim_os_name1': tf.io.FixedLenFeature([], tf.string),
        'creative_classify1': tf.io.FixedLenFeature([], tf.string),
        'total_pay_amount1':  tf.io.FixedLenFeature([], tf.float32),
         'channel1': tf.io.FixedLenFeature([], tf.string),
        'b2_sale_amt_bias':  tf.io.FixedLenFeature([], tf.int64),
         'b2_sale_amt_7d': tf.io.FixedLenFeature([], tf.int64),
         'install_time': tf.io.FixedLenFeature([], tf.string),
        'install_order_diff':  tf.io.FixedLenFeature([], tf.int64),
        'all_install_order_7d_diff':  tf.io.FixedLenFeature([], tf.int64),
        'is_a1x_a33':  tf.io.FixedLenFeature([], tf.int64),
        'platform_label':  tf.io.FixedLenFeature([], tf.string),
        'dense_vector': tf.io.FixedLenFeature([dense_vector_length], tf.float32),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    return example


def read_datafrom_tf(file):
    dataset = tf.data.TFRecordDataset(file)
    dataset = dataset.map(parse_function)
    # 将数据转换为Python字典列表
    dataset = dataset.prefetch(buffer_size=1000)
    print(dataset.take(1))
    data_list = []
    for example in tqdm_notebook(dataset):
        data_dict = {}
        for key in example.keys():
            if key in ['dense_vector', 'sale_amt_7d_label', 'sale_amt_in_1h1', 'b2_sale_amt_1d1']:
                # 将dense_vector从Tensor转换为NumPy数组
                data_dict[key] = example[key].numpy()
            elif key in ["dt", "deviceid", "install_date", "channel1", 'dim_os_name1', 'creative_classify1',
                         'adaccountid1', 'site_name1', 'install_time']:
                data_dict[key] = example[key].numpy().decode('utf-8')  # 将bytes转换为字符串
            else:
                data_dict[key] = example[key].numpy()

        data_list.append(data_dict)

    return data_list


from multiprocessing import Pool
import psutil

dense_vector_length = 121

train_file_dir = None

file_name = 'ltv_0610_user_sample_to_158'

save_path = './data/ltv_predict_sample_obj'

train_file_dir = None
all_train_dt = []
data_path = f'./data/{file_name}/'
filenames = os.listdir(data_path)
for filename in filenames:
    if "part" in filename:
        train_file_dir = os.path.join(data_path, filename)
        all_train_dt.append(train_file_dir)

print(all_train_dt)
with Pool(1) as p:
    results = p.map(read_datafrom_tf, all_train_dt[1:2])

# 转成 pandas
df = None
for res in results:
    if df is None:
        df = pd.DataFrame(res)
    else:
        df = df.append(res)

df = df.reset_index(drop=True)
df.to_pickle('./data/df_all.pkl')


['./data/ltv_0610_user_sample_to_158/part-r-00000', './data/ltv_0610_user_sample_to_158/part-r-00001', './data/ltv_0610_user_sample_to_158/part-r-00002', './data/ltv_0610_user_sample_to_158/part-r-00003', './data/ltv_0610_user_sample_to_158/part-r-00004', './data/ltv_0610_user_sample_to_158/part-r-00005', './data/ltv_0610_user_sample_to_158/part-r-00006', './data/ltv_0610_user_sample_to_158/part-r-00007', './data/ltv_0610_user_sample_to_158/part-r-00008', './data/ltv_0610_user_sample_to_158/part-r-00009']


In [3]:
def parse_function(serialized_example):
    feature_description = {
        'deviceid': tf.io.FixedLenFeature([], tf.string),
        'install_date': tf.io.FixedLenFeature([], tf.string),
        'dim_os_name1': tf.io.FixedLenFeature([], tf.string),
        'creative_classify1': tf.io.FixedLenFeature([], tf.string),
        'total_pay_amount1':  tf.io.FixedLenFeature([], tf.float32),
         'channel1': tf.io.FixedLenFeature([], tf.string),
        'b2_sale_amt_bias':  tf.io.FixedLenFeature([], tf.int64),
         'b2_sale_amt_7d': tf.io.FixedLenFeature([], tf.int64),
         'install_time': tf.io.FixedLenFeature([], tf.string),
        'install_order_diff':  tf.io.FixedLenFeature([], tf.int64),
        'all_install_order_7d_diff':  tf.io.FixedLenFeature([], tf.int64),
        'is_a1x_a33':  tf.io.FixedLenFeature([], tf.int64),
        'platform_label':  tf.io.FixedLenFeature([], tf.string),
        'dense_vector': tf.io.FixedLenFeature([dense_vector_length], tf.float32),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    return example


train_file_dir = None
dense_vector_length = 121
file_name = 'ltv_0610_user_sample_to_158'
        
def read_datafrom_tf(file):
    dataset = tf.data.TFRecordDataset(file)
    dataset = dataset.map(parse_function)
    # 将数据转换为Python字典列表
    dataset = dataset.prefetch(buffer_size=1000)

    data_list = []
    for example in tqdm_notebook(dataset):
        data_dict = {}
        for key in example.keys():
            if key in ['dense_vector', 'sale_amt_7d_label','sale_amt_in_1h1','b2_sale_amt_1d1']  :
                # 将dense_vector从Tensor转换为NumPy数组
                data_dict[key] = example[key].numpy()
            elif key in ["dt","deviceid", "install_date", "channel1",'dim_os_name1','creative_classify1','adaccountid1','site_name1','install_time']:
                data_dict[key] = example[key].numpy().decode('utf-8')  # 将bytes转换为字符串
            else:
                data_dict[key] = example[key].numpy()

        data_list.append(data_dict)

    return data_list
    

from multiprocessing import Pool
import psutil
save_path = './data/ltv_predict_sample_obj'

train_file_dir = None
all_train_dt = []
data_path = f'./data/{file_name}/'
filenames = os.listdir(data_path)
for filename in filenames:
    if "part" in filename:
        train_file_dir = os.path.join(data_path, filename)
        all_train_dt.append(train_file_dir)

print(all_train_dt)
with Pool(11) as p:
    results = p.map(read_datafrom_tf, all_train_dt)
    
# 转成 pandas
df = None
for res in results:
    if df is None:
        df = pd.DataFrame(res)
    else:
        df = df.append(res)

df = df.reset_index(drop=True)


['./data/ltv_0610_user_sample_to_158/part-r-00002', './data/ltv_0610_user_sample_to_158/part-r-00004', './data/ltv_0610_user_sample_to_158/part-r-00005', './data/ltv_0610_user_sample_to_158/part-r-00009', './data/ltv_0610_user_sample_to_158/part-r-00007', './data/ltv_0610_user_sample_to_158/part-r-00008', './data/ltv_0610_user_sample_to_158/part-r-00001', './data/ltv_0610_user_sample_to_158/part-r-00000', './data/ltv_0610_user_sample_to_158/part-r-00003', './data/ltv_0610_user_sample_to_158/part-r-00006']


2025-06-15 16:57:31.400083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-15 16:57:31.400299: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-15 16:57:31.400259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-15 16:57:31.400404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sh1-plf-ml-02): /proc/driver/nvidia/version does not exist
2025-06-15 16:57:31.400439: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-15 16:57:31.400516: I tensorflow/

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
df

In [6]:
df.rename(columns = {'channel1':'channel', 
                     'creative_classify1':'creative_classify',
                     'dim_os_name1':'dim_os_name',
                     'b2_sale_amt_1h':'b2_sale_amt_1h',
                    'channel1':'channel',
                     'site_name1':'site_name',
                    'b2_sale_amt_1d1':'b2_sale_amt_1d'
                   }, inplace = True)


df['dim_os_name'] = df['dim_os_name'].apply(lambda x: 'android'  if x == "harmonyos" else x )

## tweedie

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from lightgbm import early_stopping, log_evaluation

def train_model(train_data, y,  objective, feature_index):
    X = np.vstack(train_data.dense_vector.values)[:, feature_index]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

    if objective == "_base_":
        model = lgb.LGBMRegressor(
            objective='tweedie',
            metric = 'rmse',
            tweedie_variance_power = 1.5,
            n_estimators=500,
            learning_rate=0.05,
            max_depth = 5,
            n_jobs = 31,
            categorical_feature="0,1,2,3,4,5,6,7,8,9,10,11,12,13"
        
        )
    
    if objective ==  "_iter_":
        model = lgb.LGBMRegressor(
            objective='tweedie',
            metric = 'rmse',
            tweedie_variance_power = 1.5,
            n_estimators=500,
            learning_rate=0.05,
            max_depth = 5,
            n_jobs = 31,
            # iter_features.txt
            categorical_feature="0,1,2,3,4,5,6,7,8,9,10,118,119,120,121,122,123,124"
            # remove creative
            # categorical_feature="0,1,2,3,4,5,6,7,8,9,10,11,119,120,121,122,123"
            
            # remove city
            # categorical_feature="0,1,2,3,4,5,6,7,8,9,10,118,119,120,121,122"
        
        )
        


    model.fit(X_train, y_train, eval_set=[(X_test, y_test)],eval_metric = 'rmse', callbacks=[early_stopping(30), log_evaluation(100)])
    
    return model



def train_step(data, feauture_file ):
    res_csv = None
    folder_name = 'deep_res'
    Pred_Days = 15
    Pred_Date = '2025-03-03'
    train_days = 13 
    suffix_feature = "_base_" if "base" in feauture_file else "_iter_"
    base_indexs, feature_names = read_feature_config(file= feauture_file )
    
    if 'base' in suffix_feature:
        data['label_sale_amt'] = data['b2_label_sale_amt_1h']
    else:
        data['label_sale_amt'] = data['b2_label_sale_amt_1d']
    
    for delta in range(0, Pred_Days):
        delta_sample_days = train_days
        today = get_delta_date_str(Pred_Date, delta)
        startdate, enddate, testdate = get_delta_date_str(today, -train_days - 7), get_delta_date_str(today, -7), today
        
        train_data, test_data = get_train_test_data(data, startdate, enddate, testdate)
        base_indexs, feature_names = read_feature_config(file=feauture_file)

        model = train_model(train_data, train_data.label_sale_amt,suffix_feature, base_indexs)
        
        test_data['tweedie_predict_delta_7d'] =  model.predict(np.vstack(test_data.dense_vector.values)[:, base_indexs])

        if 'base' in suffix_feature:
            test_data['tweedie_predict_delta_7d'] = test_data['tweedie_predict_delta_7d'] - test_data['b2_sale_amt_bias'] + test_data['b2_sale_amt_1h']
        else:
            test_data['tweedie_predict_delta_7d'] = test_data['tweedie_predict_delta_7d'] - test_data['b2_sale_amt_bias'] + test_data['b2_sale_amt_1d']
            
        res_csv = pd.concat([res_csv, test_data], ignore_index=True)
        
        
        p_ltv = test_data[test_data.channel == '今日头条2.0']['tweedie_predict_delta_7d'].sum()
        t_ltv = test_data[test_data.channel == '今日头条2.0']['b2_sale_amt_7d'].sum()
        if delta == Pred_Days - 3 :
            show_lgb_top_features_v2(model, feature_names, 120, suffix_feature + 'feature_importtance.csv')
        print("=" * 60)
        print(today,":  头条总LTV bias  ",  "p_ltv: ", p_ltv,  "t_ltv: ", t_ltv, "bias: ", round((p_ltv - t_ltv) / p_ltv, 5))
        
        
    return res_csv

# online_版本 不动。作为base

In [10]:
res_online_bias = train_step(df.copy(), "feature_config/base_features.txt")

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 219.655
[200]	valid_0's rmse: 219.481
Early stopping, best iteration is:
[189]	valid_0's rmse: 219.469
2025-03-03 :  头条总LTV bias   p_ltv:  832944.742462524 t_ltv:  829551 bias:  0.00407
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 215.631
[200]	valid_0's rmse: 215.309
Early stopping, best iteration is:
[212]	valid_0's rmse: 215.291
2025-03-04 :  头条总LTV bias   p_ltv:  737066.3929671918 t_ltv:  719593 bias:  0.02371
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 214.458
[200]	valid_0's rmse: 214.177
Early stopping, best iteration is:
[245]	valid_0's rmse: 214.143
2025-03-05 :  头条总LTV bias   p_ltv:  692695.7761269368 t_ltv:  703069 bias:  -0.01498
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 210.886
[200]	valid_0's rmse: 210.65
Early stopping, best iteration is:
[243]	valid_0's rmse: 210.611
2025-

# 迭代的版本 统一叫做 iter

In [11]:
res_optim_bias = train_step(df.copy(), "feature_config/iter_features.txt")

[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 188.581
[200]	valid_0's rmse: 188.543
Early stopping, best iteration is:
[171]	valid_0's rmse: 188.534
2025-03-03 :  头条总LTV bias   p_ltv:  800185.076946677 t_ltv:  829551 bias:  -0.0367
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 183.712
[200]	valid_0's rmse: 183.621
Early stopping, best iteration is:
[201]	valid_0's rmse: 183.621
2025-03-04 :  头条总LTV bias   p_ltv:  727380.5318770427 t_ltv:  719593 bias:  0.01071
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 181.565
Early stopping, best iteration is:
[156]	valid_0's rmse: 181.533
2025-03-05 :  头条总LTV bias   p_ltv:  68423

### 优化特征 

In [40]:
# res_optim_bias = train_step(df.copy(), "feature_config/iter_features_remove_city.txt")

[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 214.706
[200]	valid_0's rmse: 214.487
Early stopping, best iteration is:
[239]	valid_0's rmse: 214.469
2025-03-03 :  头条总LTV bias   p_ltv:  790201.6434573946 t_ltv:  829551 bias:  -0.0498
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 213.611
[200]	valid_0's rmse: 213.237
[300]	valid_0's rmse: 213.115
[400]	valid_0's rmse: 213.053
Early stopping, best iteration is:
[370]	valid_0's rmse: 213.043
2025-03-04 :  头条总LTV bias   p_ltv:  699004.1899267528 t_ltv:  719593 bias:  -0.02945
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 213.596
[200]	valid_0's rmse: 213.284
Early stopping

##  合并结果 

In [12]:
channel_site_bias_compare, dim_os_creative_bias = merge_results(res_online_bias, res_optim_bias)

In [13]:
# 过滤渠道，同时从小到达排序 
df_grouped = df.groupby('channel', as_index=False).agg({'b2_sale_amt_7d': 'sum'})
df_filtered = df_grouped.query('b2_sale_amt_7d >= 100000').sort_values(by = 'b2_sale_amt_7d', ascending = False)
channls = df_filtered.channel.unique()
dts = len(df.install_date.unique())
df_filtered['mean_b2_sale_amt_7d'] = df_filtered['b2_sale_amt_7d'] / dts

### 汇总channel_x_site_weight_bias

In [15]:
summary_res = None
for cl in channls:
    print("当前测试的渠道：",cl)
    temp_res = get_channel_bias(channel_site_bias_compare, channel =[cl]).reset_index()
    temp_res['channelXsite_name_weight_bias'] = cl
    temp_res['every_dt_7days_b2_sale_mean'] = round(df_filtered.loc[df_filtered['channel'] == cl, 'mean_b2_sale_amt_7d'].values[0], 0)
    summary_res = pd.concat([summary_res, temp_res])
    display(temp_res)

save_excel_with_progress_bar(summary_res, "channelXsite_name_weight_bias", "channelXsite_name_weight_bias.xlsx")

当前测试的渠道： appstore
========== 加权bias ==========


,base_bias,iter_bias,提升
2025-03-03,0.0676,0.0486,-0.2806
2025-03-04,0.0821,0.0486,-0.4077
2025-03-05,0.0240,0.0137,-0.4281
2025-03-06,0.1063,0.0440,-0.5853
2025-03-07,0.0874,0.0462,-0.4712
2025-03-08,0.0672,0.0433,-0.3543
2025-03-09,0.0150,0.0043,-0.7068
2025-03-10,0.0608,0.0357,-0.4121
2025-03-11,0.0834,0.0246,-0.7043
2025-03-12,0.0187,0.0057,-0.6936


base_bias   0.0572
iter_bias   0.0294
dtype: float64

,index,base_bias,iter_bias,提升,channelXsite_name_weight_bias,every_dt_7days_b2_sale_mean
0,2025-03-03,0.0676,0.0486,-0.2806,appstore,1026265.0000
1,2025-03-04,0.0821,0.0486,-0.4077,appstore,1026265.0000
2,2025-03-05,0.0240,0.0137,-0.4281,appstore,1026265.0000
3,2025-03-06,0.1063,0.0440,-0.5853,appstore,1026265.0000
4,2025-03-07,0.0874,0.0462,-0.4712,appstore,1026265.0000
5,2025-03-08,0.0672,0.0433,-0.3543,appstore,1026265.0000
6,2025-03-09,0.0150,0.0043,-0.7068,appstore,1026265.0000
7,2025-03-10,0.0608,0.0357,-0.4121,appstore,1026265.0000
8,2025-03-11,0.0834,0.0246,-0.7043,appstore,1026265.0000
9,2025-03-12,0.0187,0.0057,-0.6936,appstore,1026265.0000


当前测试的渠道： 今日头条2.0
========== 加权bias ==========


...

当前测试的渠道： 2024年春节裂变活动
========== 加权bias ==========


,base_bias,iter_bias,提升
2025-03-03,0.7451,0.5129,-0.3115
2025-03-04,0.2719,0.2596,-0.0452
2025-03-05,0.1377,0.4000,1.9039
2025-03-06,0.2449,0.4583,0.8709
2025-03-07,1.1439,0.6159,-0.4615
2025-03-08,0.8030,0.4070,-0.4930
2025-03-09,0.8649,0.5408,-0.3747
2025-03-10,0.2184,0.1953,-0.1056
2025-03-11,0.0073,0.1724,22.4712
2025-03-12,1.1953,0.2991,-0.7497


base_bias   0.8026
iter_bias   0.4961
dtype: float64

,index,base_bias,iter_bias,提升,channelXsite_name_weight_bias,every_dt_7days_b2_sale_mean
0,2025-03-03,0.7451,0.5129,-0.3115,2024年春节裂变活动,4861.0000
1,2025-03-04,0.2719,0.2596,-0.0452,2024年春节裂变活动,4861.0000
2,2025-03-05,0.1377,0.4000,1.9039,2024年春节裂变活动,4861.0000
3,2025-03-06,0.2449,0.4583,0.8709,2024年春节裂变活动,4861.0000
4,2025-03-07,1.1439,0.6159,-0.4615,2024年春节裂变活动,4861.0000
5,2025-03-08,0.8030,0.4070,-0.4930,2024年春节裂变活动,4861.0000
6,2025-03-09,0.8649,0.5408,-0.3747,2024年春节裂变活动,4861.0000
7,2025-03-10,0.2184,0.1953,-0.1056,2024年春节裂变活动,4861.0000
8,2025-03-11,0.0073,0.1724,22.4712,2024年春节裂变活动,4861.0000
9,2025-03-12,1.1953,0.2991,-0.7497,2024年春节裂变活动,4861.0000


Excel 文件已保存: channelXsite_name_weight_bias.xlsx


### 汇总OSx creative classfy weight bias

In [16]:
summary_res = None
for cl in channls:
    print("当前测试的渠道：",cl)
    temp_res = get_channel_bias(dim_os_creative_bias, channel =[cl]).reset_index()
    temp_res['channelXos_biz_weight_bias'] = cl
    temp_res['every_dt_7days_b2_sale_mean'] = round(df_filtered.loc[df_filtered['channel'] == cl, 'mean_b2_sale_amt_7d'].values[0], 0)
    summary_res = pd.concat([summary_res, temp_res])
    display(temp_res)
save_excel_with_progress_bar(summary_res, "channelXos_biz_weight_bias", "dim_os_creative_bias.xlsx")

当前测试的渠道： appstore
========== 加权bias ==========


,base_bias,iter_bias,提升
2025-03-03,0.0680,0.0490,-0.2790
2025-03-04,0.0820,0.0490,-0.4019
2025-03-05,0.0240,0.0140,-0.4149
2025-03-06,0.1060,0.0440,-0.5844
2025-03-07,0.0870,0.0460,-0.4707
2025-03-08,0.0670,0.0430,-0.3577
2025-03-09,0.0150,0.0040,-0.7285
2025-03-10,0.0610,0.0360,-0.4092
2025-03-11,0.0830,0.0250,-0.6980
2025-03-12,0.0190,0.0060,-0.6806


base_bias   0.0572
iter_bias   0.0295
dtype: float64

,index,base_bias,iter_bias,提升,channelXos_biz_weight_bias,every_dt_7days_b2_sale_mean
0,2025-03-03,0.0680,0.0490,-0.2790,appstore,1026265.0000
1,2025-03-04,0.0820,0.0490,-0.4019,appstore,1026265.0000
2,2025-03-05,0.0240,0.0140,-0.4149,appstore,1026265.0000
3,2025-03-06,0.1060,0.0440,-0.5844,appstore,1026265.0000
4,2025-03-07,0.0870,0.0460,-0.4707,appstore,1026265.0000
5,2025-03-08,0.0670,0.0430,-0.3577,appstore,1026265.0000
6,2025-03-09,0.0150,0.0040,-0.7285,appstore,1026265.0000
7,2025-03-10,0.0610,0.0360,-0.4092,appstore,1026265.0000
8,2025-03-11,0.0830,0.0250,-0.6980,appstore,1026265.0000
9,2025-03-12,0.0190,0.0060,-0.6806,appstore,1026265.0000


当前测试的渠道： 今日头条2.0
========== 加权bias ==========


...

当前测试的渠道： 2024年春节裂变活动
========== 加权bias ==========


,base_bias,iter_bias,提升
2025-03-03,0.7448,0.5130,-0.3111
2025-03-04,0.2717,0.2599,-0.0434
2025-03-05,0.5233,0.4607,-0.1197
2025-03-06,0.6249,0.6933,0.1094
2025-03-07,1.1434,0.6162,-0.4611
2025-03-08,0.8028,0.4069,-0.4931
2025-03-09,0.8621,0.5387,-0.3752
2025-03-10,0.2184,0.1955,-0.1049
2025-03-11,0.7953,0.5738,-0.2784
2025-03-12,1.1949,0.2986,-0.7500


base_bias   0.9703
iter_bias   0.5568
dtype: float64

,index,base_bias,iter_bias,提升,channelXos_biz_weight_bias,every_dt_7days_b2_sale_mean
0,2025-03-03,0.7448,0.5130,-0.3111,2024年春节裂变活动,4861.0000
1,2025-03-04,0.2717,0.2599,-0.0434,2024年春节裂变活动,4861.0000
2,2025-03-05,0.5233,0.4607,-0.1197,2024年春节裂变活动,4861.0000
3,2025-03-06,0.6249,0.6933,0.1094,2024年春节裂变活动,4861.0000
4,2025-03-07,1.1434,0.6162,-0.4611,2024年春节裂变活动,4861.0000
5,2025-03-08,0.8028,0.4069,-0.4931,2024年春节裂变活动,4861.0000
6,2025-03-09,0.8621,0.5387,-0.3752,2024年春节裂变活动,4861.0000
7,2025-03-10,0.2184,0.1955,-0.1049,2024年春节裂变活动,4861.0000
8,2025-03-11,0.7953,0.5738,-0.2784,2024年春节裂变活动,4861.0000
9,2025-03-12,1.1949,0.2986,-0.7500,2024年春节裂变活动,4861.0000


Excel 文件已保存: dim_os_creative_bias.xlsx


In [17]:
send_filelist("离线在线版特征", ["dim_os_creative_bias.xlsx"])
send_filelist("离线在线版特征", ["channelXsite_name_weight_bias.xlsx"])

邮件发送成功
邮件发送成功


In [53]:
toutiao_channel_site.head()

,install_date,channel,site_name,tweedie_predict_delta_7d,b2_sale_amt_7d,tweedie_predict_delta_7d_base,tweedie_predict_delta_7d_iter,iter_bias,base_bias
51,2025-03-03,今日头条2.0,None,39933.8621,31188,39933.8621,33130.0779,0.0623,0.2804
53,2025-03-03,今日头条2.0,"优量汇,",522.2537,268,522.2537,476.1940,0.7740,0.9452
56,2025-03-03,今日头条2.0,"头条信息流,抖音信息流,",299624.8560,312426,299624.8560,290132.5780,-0.0714,-0.0410
57,2025-03-03,今日头条2.0,"头条信息流,抖音信息流,通投智选,",32776.1394,18798,32776.1394,25800.1213,0.3725,0.7436
58,2025-03-03,今日头条2.0,"头条信息流,西瓜信息流,抖音信息流,",32571.0395,46419,32571.0395,31080.3880,-0.3304,-0.2983


### 单独看下头条2.0

In [18]:
toutiao_channel_site = channel_site_bias_compare[(channel_site_bias_compare.channel == '今日头条2.0')]
save_excel_with_progress_bar(toutiao_channel_site, "site_name", "toutiao_Xos_biz_weight_bias.xlsx")
send_filelist("toutiao2.0", ["toutiao_Xos_biz_weight_bias.xlsx"])

Excel 文件已保存: toutiao_Xos_biz_weight_bias.xlsx
邮件发送成功


In [19]:
toutiao_channel_site_by_dt_res, toutiao_channel_site_summary_res = get_channel_site_bias(toutiao_channel_site)

In [20]:
save_excel_with_progress_bar(toutiao_channel_site_by_dt_res, 'site_name', "toutiao_channel_site_by_dt_res.xlsx")
save_excel_with_progress_bar(toutiao_channel_site_summary_res, 'site_name', "toutiao_channel_site_summary_res.xlsx")

Excel 文件已保存: toutiao_channel_site_by_dt_res.xlsx
Excel 文件已保存: toutiao_channel_site_summary_res.xlsx


In [21]:
send_filelist("只看头条 加上城市id", ["toutiao_channel_site_by_dt_res.xlsx"])
send_filelist("只看头条加上城市id", ["toutiao_channel_site_summary_res.xlsx"])

邮件发送成功
邮件发送成功


In [22]:

toutiao_dim_os_creative_bias_by_dt_res, toutiao_dim_os_creative_bias_summary_res = get_channel_biz_os_bias(dim_os_creative_bias)


In [23]:
save_excel_with_progress_bar(toutiao_dim_os_creative_bias_by_dt_res, 'creative_classify', "toutiao_dim_os_creative_bias_by_dt_res.xlsx")
save_excel_with_progress_bar(toutiao_dim_os_creative_bias_summary_res, 'creative_classify', "toutiao_dim_os_creative_bias_summary_res.xlsx")

Excel 文件已保存: toutiao_dim_os_creative_bias_by_dt_res.xlsx
Excel 文件已保存: toutiao_dim_os_creative_bias_summary_res.xlsx


In [24]:
send_filelist("只看头条", ["toutiao_dim_os_creative_bias_by_dt_res.xlsx", "toutiao_dim_os_creative_bias_summary_res.xlsx"])

邮件发送成功


In [25]:
get_channel_biz_os_bias??

Signature:
get_channel_biz_os_bias(
    res,
    thd=-1,
    base_col='tweedie_predict_delta_7d_base',
    iter_col='tweedie_predict_delta_7d_iter',
    channel=['今日头条2.0'],
)
Docstring: <no docstring>
Source:   
def get_channel_biz_os_bias(res, thd = -1, base_col = 'tweedie_predict_delta_7d_base', iter_col = 'tweedie_predict_delta_7d_iter', channel = ['今日头条2.0']):
    res = res[res.channel.isin(channel)]
    
    if thd >= 0:
        res = res[res.b2_sale_amt_7d >= thd]
    res = res[(res.base_bias.isna() == False) & (res.iter_bias.isna() == False)]

    res['iter_bias'] = res['iter_bias'].apply(lambda x: abs(x))
    res['base_bias'] = res['base_bias'].apply(lambda x: abs(x))
    
    temp = res.groupby(['dim_os_name','creative_classify']).agg({'b2_sale_amt_7d':'sum'})
    res = res.merge(temp.rename(columns={'b2_sale_amt_7d': 'b2_sale_amt_7d_sum'}), 
    on=['dim_os_name','creative_classify'], 
    how='left'
    )

    
    res['提升'] = (res['iter_bias'] - res['base_bias']) / (res['b